In [1]:
%load_ext autoreload
%autoreload 2
import gym
import os
import numpy as np
import random
import torch
from experiment import rollout, ReplayBuffer, Trajectory, load_model, save_model, Behavior
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

In [2]:
env = gym.make('LunarLander-v2')

In [3]:
rb = ReplayBuffer(max_size=500, last_few=200)

# Random rollout
trajectories, avg_reward = rollout(episodes=100, env=env, render=False)
rb.add(trajectories)

print(f"Average Episode Reward: {avg_reward}")

Average Episode Reward: -181.43609374148468


In [4]:
loss_object = torch.nn.CrossEntropyLoss().to(device)
model = Behavior(input_shape=env.observation_space.shape[0]+2, num_actions=env.action_space.n).to(device)
optimizer = torch.optim.Adam(model.parameters())

In [5]:
batch_size = 1024

In [6]:
def train_step(inputs, targets):
    optimizer.zero_grad()    
    predictions = model(inputs)
    loss = loss_object(predictions, targets)
    
    loss.backward()
    optimizer.step()
    
    return loss

def action_fn(model, inputs, sample_action=True):
    action_probs = model(inputs)
    action_probs = torch.sigmoid(action_probs)

    if sample_action:
        m = torch.distributions.categorical.Categorical(probs=action_probs)            
        action = int(m.sample().squeeze().cpu().numpy())
    else:
        action = int(np.argmax(action_probs.detach().squeeze().numpy()))
    return action
    

In [7]:
loss_sum = 0
loss_count = 0


In [ ]:
epochs = 1000000
epoch, model, optimizer, loss = load_model('lunar_lander', model, optimizer, device, train=True)

for i in range(epoch, epochs+epoch):
    x, y = rb.sample(batch_size, device)    
    loss = train_step(x, y)
    loss_sum += loss
    loss_count += 1
    
    if i % 100 == 0:        
        trajectories, mean_reward = rollout(100, env=env, model=model, sample_action=False, replay_buffer=rb, 
                              device=device, action_fn=action_fn)
        rb.add(trajectories)
        
        print(f"Average Episode Reward: {mean_reward}")        

    if i % 200 == 0:
        print(f'i: {i}, Loss: {loss_sum/loss_count}') #'\t Accuracy: {accuracy_m.result()}')
        save_model('lunar_lander', i, model, optimizer, loss)

No checkpoint found. Creating new model.
Average Episode Reward: -319.6817528311101
i: 0, Loss: 1.4287989139556885


In [27]:
rb.sample_command()

(111.275, 17.692855727918)

In [33]:
cmd = (500, 500) #rb.sample_command()
env = gym.make('LunarLander-v2')
e, model, _, l = load_model(name='lunar_lander', train=False, model=model, optimizer=optimizer, device=device)
print(f"Loaded model at epoch: {e} with loss {l}")
_, mean_reward = rollout(episodes=3, env=env, model=model, sample_action=False, 
                      replay_buffer=rb, render=True, device=device, action_fn=action_fn)


print(f"Average Episode Reward: {mean_reward}")

Existing model found. Loading.
Loaded model at epoch: 9000 with loss 1.2257001399993896
Average Episode Reward: -234.4207812771572
